In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from PIL import Image, ImageOps
import sys
import re
from sklearn.model_selection import GroupShuffleSplit
import os
from tqdm import tqdm
import nibabel as nib
import imageio

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)


# Preprocess CSV

In [4]:
data_df = pd.read_csv('/vast/amr10211/deep_learning_final_project/train.csv')
meta = pd.read_csv('/vast/amr10211/deep_learning_final_project/train_series_meta.csv')


In [5]:
data_df = data_df.merge(meta, on='patient_id', how='left')

In [6]:
# #Create liver classificiation column - MULTICLASS
# data_df['target'] = 0
# data_df.loc[data_df['liver_healthy'] == 1, 'target'] = 0
# data_df.loc[data_df['liver_low'] == 1, 'target'] = 1
# data_df.loc[data_df['liver_high'] == 1, 'target'] = 2

In [7]:
#Create liver classificiation column - BINARY
data_df['target'] = 0
data_df.loc[data_df['liver_healthy'] == 1, 'target'] = 0
data_df.loc[data_df['liver_low'] == 1, 'target'] = 1
data_df.loc[data_df['liver_high'] == 1, 'target'] = 1

In [8]:
#Add image name:
data_df['img_id'] = data_df['patient_id'].astype(str) + '_' + data_df['series_id'].astype(str)
data_df['img_name'] = data_df['patient_id'].astype(str) + '_' + data_df['series_id'].astype(str) + '.png'
data_df['preprocessed_img_name'] = data_df['patient_id'].astype(str) + '_' + data_df['series_id'].astype(str) + '.npy'
data_df['preprocessed_segm_name'] = data_df['series_id'].astype(str) + '.npy'


In [9]:
#Select only columns needed for this project
data_df = data_df[['patient_id', 'series_id', 'target', 'img_id', 'img_name', 'preprocessed_img_name', 'preprocessed_segm_name']]

In [10]:
data_df['target'].value_counts()

target
0    4236
1     475
Name: count, dtype: int64

# Split Data

In [11]:
#Split data with no leakage of same patient in test, val, and train:
group_shuffle_split_80_20 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=7)
group_shuffle_split_50_50 = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=7)

for train_index, temp_index in group_shuffle_split_80_20.split(data_df, groups=data_df['patient_id']):
    train_df = data_df.iloc[train_index]
    temp_df = data_df.iloc[temp_index]

for val_index, test_index in  group_shuffle_split_50_50.split(temp_df, groups=temp_df['patient_id']):
    val_df = temp_df.iloc[val_index]
    test_df = temp_df.iloc[test_index]

In [12]:
#Check for leakage:
train_image_ids_set = set(train_df['patient_id'])
val_image_ids_set = set(val_df['patient_id'])
test_image_ids_set = set(test_df['patient_id'])

intersection_train_val = train_image_ids_set.intersection(val_image_ids_set)
intersection_train_test = train_image_ids_set.intersection(test_image_ids_set)
intersection_val_test = val_image_ids_set.intersection(test_image_ids_set)

# Check if there are any matching patient_ids between the sets
if intersection_train_val:
    print("Matching patient_id between train_set and val_set:", intersection_train_val)

if intersection_train_test:
    print("Matching patient_id between train_set and test_set:", intersection_train_test)

if intersection_val_test:
    print("Matching patient_id between val_set and test_set:", intersection_val_test)
else:
    print("No matching patient_id found between any sets.")

No matching patient_id found between any sets.


In [13]:
#Save to CSV - BINARY:
data_df.to_csv('full_data_binary.csv')
train_df.to_csv('train_binary.csv')
val_df.to_csv('val_binary.csv')
test_df.to_csv('test_binary.csv')

In [14]:
# #Make small datasources for model buidling - BINARY:
train_sample = train_df.sample(frac=0.05, random_state=7)
val_sample = val_df.sample(frac=0.05, random_state=7)
test_sample = test_df.sample(frac=0.05, random_state=7)

train_sample.to_csv('train_sample_binary.csv')
val_sample.to_csv('val_sample_binary.csv')
test_sample.to_csv('test_sample_binary.csv')

In [11]:
# #Save to CSV - MULTICLASS:
# data_df.to_csv('full_data.csv')
# train_df.to_csv('train.csv')
# val_df.to_csv('val.csv')
# test_df.to_csv('test.csv')

In [12]:
# #Make small datasources for model buidling - MULTICLASS:
# train_sample = train_df.sample(frac=0.05, random_state=7)
# val_sample = val_df.sample(frac=0.05, random_state=7)
# test_sample = test_df.sample(frac=0.05, random_state=7)

# train_sample.to_csv('train_sample.csv')
# val_sample.to_csv('val_sample.csv')
# test_sample.to_csv('test_sample.csv')

# Preprocess Segmentations and Images

### Segmentation Preprocessing from 1st place Kaggle Winners 
https://github.com/Nischaydnk/RSNA-2023-1st-place-solution/blob/main/Datasets/make_segmentation_data1.py

In [193]:
# def load_total_segmentation_volume(path):
#     volume = nib.load(path).get_fdata()
    
#     volume = volume[::-1, ::-1, ::-1].transpose(2, 1, 0)
    
#     return volume

In [194]:
def load_total_segmentation_volume(path):
    volume = nib.load(path).get_fdata()
    
    # Flip along each axis
    volume = volume[:, :, ::-1].transpose(2, 1, 0)  # Flip along Z-axis, then transpose
    volume = volume[:, ::-1, :]  # Flip along Y-axis
    volume = volume[::-1, :, :]  # Flip along X-axis
    
    return volume

In [195]:
image_names = os.listdir('/vast/amr10211/deep_learning_final_project/images')
image_names = [filename for filename in image_names if filename.endswith(('.jpg', '.jpeg', '.png', '.gif', '.bmp'))]

segm_names = os.listdir('/vast/amr10211/deep_learning_final_project/segmentations/')
segm_names = [int(filename[:-4]) for filename in segm_names if filename.endswith(('.nii'))]


In [196]:
image_df = pd.DataFrame({'img_slice_name': image_names})
image_df['series_id'] = image_df['img_slice_name'].apply(lambda x: x.split('_')[1])
image_df['series_id'] = pd.to_numeric(image_df['series_id'])
# img_slice_name['slice
image_df = image_df.merge(data_df, on='series_id', how='left')

In [197]:
segm_df = data_df[data_df['series_id'].isin(segm_names)].reset_index(drop=True)
segm_df['segm_name']= segm_df['series_id'].astype(str) + '.nii'
segm_df['preprocessed_segm_name']= segm_df['series_id'].astype(str) + '.npy'


In [181]:
# #UNCOMMENT TO USE:

# #Format segmentations into 3D numpy arrays:
# segm_path = '/vast/amr10211/deep_learning_final_project/segmentations/'
# preprocessed_segm_path = '/vast/amr10211/deep_learning_final_project/segm_preprocessed/'
# segm_file_names = list(segm_df['segm_name'].unique())

# for segm_file in tqdm(segm_file_names, total=len(segm_file_names)):
#     segm = load_total_segmentation_volume(os.path.join(segm_path, segm_file))
#     segm = np.array(segm)

#     #Only include liver segmentations (for this project):
#     segm = np.where(segm == 1, 1, 0)
#     segm = segm[np.arange(0, segm.shape[0], 2)]

#     segm_name = str(segm_file[:-4]) + '.npy'
    
#     output_filename = os.path.join(preprocessed_segm_path, segm_name)

#     np.save(output_filename, segm)
    

100%|██████████| 206/206 [07:11<00:00,  2.10s/it]


In [86]:
# #UNCOMMENT TO USE:

# #Format images into 3D numpy arrays:
# img_path = '/vast/amr10211/deep_learning_final_project/images/'
# preprocessed_img_path = '/vast/amr10211/deep_learning_final_project/images_preprocessed/'

# for index, row in tqdm(data_df.iterrows(), total=len(data_df)):
#     img_name = row['img_name']
#     preprocessed_img_name =  row['preprocessed_img_name']
#     img_slice_names = image_df[image_df['img_name']==img_name]['img_slice_name']
#     img = []
#     for img_slice in img_slice_names[::2]:  #EVERY OTHER IMAGE SLICE
#         img.append(np.array(Image.open(os.path.join(img_path, img_slice))))
#     img = np.array(img)
        
#     output_filename = os.path.join(preprocessed_img_path, preprocessed_img_name)

#     np.save(output_filename, img)
    

100%|██████████| 4711/4711 [1:06:51<00:00,  1.17it/s]


In [198]:
# Filter CSVs for those with segmentations:
data_segm_df = data_df[data_df['preprocessed_segm_name'].isin(segm_df['preprocessed_segm_name'])]
train_segm_df = train_df[train_df['preprocessed_segm_name'].isin(segm_df['preprocessed_segm_name'])]
val_segm_df = val_df[val_df['preprocessed_segm_name'].isin(segm_df['preprocessed_segm_name'])]
test_segm_df = test_df[test_df['preprocessed_segm_name'].isin(segm_df['preprocessed_segm_name'])]

data_segm_df.to_csv('full_data_segm.csv')
train_segm_df.to_csv('train_segm.csv')
val_segm_df.to_csv('val_segm.csv')
test_segm_df.to_csv('test_segm.csv')

#Make small datasources for model buidling:
train_segm_sample = train_segm_df.sample(frac=0.2, random_state=47)
val_segm_sample = val_segm_df.sample(frac=0.2, random_state=47)
test_segm_sample = test_segm_df.sample(frac=0.2, random_state=47)

train_segm_sample.to_csv('train_segm_sample.csv')
val_segm_sample.to_csv('val_segm_sample.csv')
test_segm_sample.to_csv('test_segm_sample.csv')


# Trouble Shooting Orientation:

In [16]:
# Use the NIFTI header information along with DICOM metadata to determine the appropriate orientation such that the DICOM images and segmentation match.
# You may find the 'ImageOrientationPatient' and 'ImagePositionPatient' columns useful.

train_dicom_tags = pd.read_parquet('/vast/amr10211/deep_learning_final_project/train_dicom_tags.parquet')


In [19]:
test_segm_sample

,patient_id,series_id,target,img_id,img_name,preprocessed_img_name,preprocessed_segm_name
2519,4093,8236,1,4093_8236,4093_8236.png,4093_8236.npy,8236.npy
4452,65504,50212,1,65504_50212,65504_50212.png,65504_50212.npy,50212.npy
2043,35056,49753,1,35056_49753,35056_49753.png,35056_49753.npy,49753.npy


In [20]:
train_dicom_tags['PatientPosition'].value_counts()

PatientPosition
FFS    885524
HFS    615129
Name: count, dtype: int64

In [67]:
data_df[data_df['series_id']==21057]

,patient_id,series_id,target,img_id,img_name,preprocessed_img_name,preprocessed_segm_name
0,10004,21057,0,10004_21057,10004_21057.png,10004_21057.npy,21057.npy


In [21]:
# train_dicom_tags[(train_dicom_tags['PatientID'] == '4093') & train_dicom_tags['SeriesInstanceUID'].str.contains('8236')].sort_values('InstanceNumber')

In [22]:
segm_file_names = list(segm_df['segm_name'].unique())

In [162]:
import pandas as pd

# Assuming train_dicom_tags is a DataFrame with columns 'SeriesInstanceUID', 'InstanceNumber', and 'ImagePositionPatient'

# Sort the DataFrame by 'SeriesInstanceUID' and 'InstanceNumber'
train_dicom_tags_sorted = train_dicom_tags.sort_values(['SeriesInstanceUID', 'InstanceNumber'])
series_ids_str = segm_df['series_id'].astype(str)

# Filter train_dicom_tags_sorted based on SeriesInstanceUID
train_dicom_tags_sorted = train_dicom_tags_sorted[train_dicom_tags_sorted['SeriesInstanceUID'].apply(lambda x: any(series_id in x for series_id in series_ids_str))]

# Define a function to calculate the Z value range for each group
def calculate_z_range(group):
    first_z = group['ImagePositionPatient'].iloc[0].split()[-1][:-1]  # Get Z value of the first instance and remove trailing bracket
    last_z = group['ImagePositionPatient'].iloc[-1].split()[-1][:-1]  # Get Z value of the last instance and remove trailing bracket
    return pd.Series({'FirstZ': float(first_z), 'LastZ': float(last_z)})  # Convert to float and return as Series

# Group the DataFrame by 'SeriesInstanceUID' and apply the function
z_range_per_series = train_dicom_tags_sorted.groupby('SeriesInstanceUID').apply(calculate_z_range)

# Now z_range_per_series is a DataFrame with 'FirstZ' and 'LastZ' for each 'SeriesInstanceUID'


In [163]:
z_range_per_series['B2T'] = z_range_per_series['FirstZ']>z_range_per_series['LastZ']

In [164]:
z_range_per_series[z_range_per_series['B2T']==False]['B2T']

SeriesInstanceUID
1.2.123.12345.1.2.3.55251.50282    False
Name: B2T, dtype: bool

In [66]:
train_dicom_tags.sort_values(['SeriesInstanceUID', 'InstanceNumber'])

,BitsAllocated,BitsStored,Columns,ContentDate,ContentTime,FrameOfReferenceUID,HighBit,ImageOrientationPatient,ImagePositionPatient,InstanceNumber,KVP,PatientID,PatientPosition,PhotometricInterpretation,PixelRepresentation,PixelSpacing,RescaleIntercept,RescaleSlope,Rows,SOPInstanceUID,SamplesPerPixel,SeriesInstanceUID,SeriesNumber,SliceThickness,StudyInstanceUID,WindowCenter,WindowWidth,FileMetaInformationVersion,ImplementationClassUID,ImplementationVersionName,MediaStorageSOPClassUID,MediaStorageSOPInstanceUID,TransferSyntaxUID,path,RescaleType
1450010,16,12,512,20230721,232307.516407,1.2.826.0.1.3680043.8.498.61841901354930484747...,11,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[-240.55273, -378.05273, -1186.4]",171,90.0,10004,HFS,MONOCHROME2,0,"[0.89453125, 0.89453125]",-1024.0,1.0,512,1.2.123.12345.1.2.3.10004.1.171,1,1.2.123.12345.1.2.3.10004.21057,16,1.0,1.2.123.12345.1.2.3.10004,50.0,400.0, ,1.2.3.123456.4.5.1234.1.12.0,PYDICOM 2.4.0,1.2.840.10008.5.1.4.1.1.2,1.2.123.12345.1.2.3.10004.1.171,1.2.840.10008.1.2.5,train_images/10004/21057/171.dcm,HU
1449919,16,12,512,20230721,232307.684850,1.2.826.0.1.3680043.8.498.61841901354930484747...,11,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[-240.55273, -378.05273, -1186.9]",172,90.0,10004,HFS,MONOCHROME2,0,"[0.89453125, 0.89453125]",-1024.0,1.0,512,1.2.123.12345.1.2.3.10004.1.172,1,1.2.123.12345.1.2.3.10004.21057,16,1.0,1.2.123.12345.1.2.3.10004,50.0,400.0, ,1.2.3.123456.4.5.1234.1.12.0,PYDICOM 2.4.0,1.2.840.10008.5.1.4.1.1.2,1.2.123.12345.1.2.3.10004.1.172,1.2.840.10008.1.2.5,train_images/10004/21057/172.dcm,HU
1450702,16,12,512,20230721,232307.841491,1.2.826.0.1.3680043.8.498.61841901354930484747...,11,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[-240.55273, -378.05273, -1187.4]",173,90.0,10004,HFS,MONOCHROME2,0,"[0.89453125, 0.89453125]",-1024.0,1.0,512,1.2.123.12345.1.2.3.10004.1.173,1,1.2.123.12345.1.2.3.10004.21057,16,1.0,1.2.123.12345.1.2.3.10004,50.0,400.0, ,1.2.3.123456.4.5.1234.1.12.0,PYDICOM 2.4.0,1.2.840.10008.5.1.4.1.1.2,1.2.123.12345.1.2.3.10004.1.173,1.2.840.10008.1.2.5,train_images/10004/21057/173.dcm,HU
1450760,16,12,512,20230721,232307.992407,1.2.826.0.1.3680043.8.498.61841901354930484747...,11,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[-240.55273, -378.05273, -1187.9]",174,90.0,10004,HFS,MONOCHROME2,0,"[0.89453125, 0.89453125]",-1024.0,1.0,512,1.2.123.12345.1.2.3.10004.1.174,1,1.2.123.12345.1.2.3.10004.21057,16,1.0,1.2.123.12345.1.2.3.10004,50.0,400.0, ,1.2.3.123456.4.5.1234.1.12.0,PYDICOM 2.4.0,1.2.840.10008.5.1.4.1.1.2,1.2.123.12345.1.2.3.10004.1.174,1.2.840.10008.1.2.5,train_images/10004/21057/174.dcm,HU
1450050,16,12,512,20230721,232308.142492,1.2.826.0.1.3680043.8.498.61841901354930484747...,11,"[1.0, 0.0, 0.0, 0.0, 1.0, 0.0]","[-240.55273, -378.05273, -1188.4]",175,90.0,10004,HFS,MONOCHROME2,0,"[0.89453125, 0.89453125]",-1024.0,1.0,512,1.2.123.12345.1.2.3.10004.1.175,1,1.2.123.12345.1.2.3.10004.21057,16,1.0,1.2.123.12345.1.2.3.10004,50.0,400.0, ,1.2.3.123456.4.5.1234.1.12.0,PYDICOM 2.4.0,1.2.840.10008.5.1.4.1.1.2,1.2.123.12345.1.2.3.10004.1.175,1.2.840.10008.1.2.5,train_images/10004/21057/175.dcm,HU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831168,16,12,512,20230721,232306.667612,1.2.826.0.1.3680043.8.498.74440228295669709696...,11,"[1, 0, 0, 0, 1, 0]","[-238.572265625, -411.572265625, -917.2]",1032,140.0,9983,HFS,MONOCHROME2,0,"[0.85546875, 0.85546875]",-1024.0,1.0,512,1.2.123.12345.1.2.3.9983.1.1032,1,1.2.123.12345.1.2.3.9983.10806,9,1.0,1.2.123.12345.1.2.3.9983,50.0,400.0, ,1.2.3.123456.4.5.1234.1.12.0,PYDICOM 2.4.0,1.2.840.10008.5.1.4.1.1.2,1.2.123.12345.1.2.3.9983.1.1032,1.2.840.10008.1.2.5,train_images/9983/10806/1032.dcm,HU
830743,16,12,512,20230721,232306.821970,1.2.826.0.1.3680043.8.498.74440228295669709696...,11,"[1, 0, 0, 0, 1, 0]","[-238.572265625, -411.572265625, -917.9]",1033,140.0,9983,HFS,MONOCHROME2,0,"[0.85546875, 0.85546875]",-1024.0,1.0,512,1.2.123.12345.1.2.3.9983.

In [142]:
img = np.load('/vast/amr10211/deep_learning_final_project/images_preprocessed/10004_21057.npy')
segm = np.load('/vast/amr10211/deep_learning_final_project/segm_preprocessed/21057.npy')
original_segm = nib.load('/vast/amr10211/deep_learning_final_project/segmentations/21057.nii')
original_segm = original_segm.get_fdata()
original_segm_header = nib.load('/vast/amr10211/deep_learning_final_project/segmentations/21057.nii').header
original_segm = np.array(original_segm)

In [159]:
segm_df[segm_df['series_id']==50282]

,patient_id,series_id,target,img_id,img_name,preprocessed_img_name,preprocessed_segm_name,segm_name,segm_orientation


In [113]:
print(original_segm_header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 16384
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [   3  512  512 1022    1    1    1    1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : float32
bitpix          : 32
slice_start     : 0
pixdim          : [-1.          0.89453125  0.89453125  0.5         0.          0.
  0.          0.        ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'2203.6-dirty 2022-09-08T15:38:56+01:00'
aux_file        : b''
qform_code      : scanner
sform_code      : unknown
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoff

In [106]:
# segm_path = '/vast/amr10211/deep_learning_final_project/segmentations/'
# preprocessed_segm_path = '/vast/amr10211/deep_learning_final_project/segm_preprocessed/'
# segm_file_names = list(segm_df['segm_name'].unique())

# for segm_file in tqdm(segm_file_names, total=len(segm_file_names)):
#     segm = nib.load(os.path.join(segm_path, segm_file))
#     data = segm.get_fdata()
    
#     voxel_sizes = segm.header.get_zooms()
    
#     # If voxel size along the Z-axis is positive, it's likely bottom-to-top, if negative, it's top-to-bottom
#     if voxel_sizes[2] > 0:
#         segm_df.loc[segm_df['segm_name'] == segm_file, 'segm_orientation'] = 'B2T'
#     else:
#         segm_df.loc[segm_df['segm_name'] == segm_file, 'segm_orientation'] = 'T2B'

In [107]:
# segm_df['segm_orientation'].unique()

In [108]:
# img = nib.load('/vast/amr10211/deep_learning_final_project/segmentations/21057.nii')

# # Get the image data array
# data = img.get_fdata()

# # Check the dimensions of the data array
# print("Data shape:", data.shape)
# voxel_sizes = img.header.get_zooms()
# print("Voxel sizes:", voxel_sizes)

# # If voxel size along the Z-axis is positive, it's likely bottom-to-top, if negative, it's top-to-bottom
# if voxel_sizes[2] > 0:
#     print("Likely bottom-to-top acquisition.")
# else:
#     print("Likely top-to-bottom acquisition.")

In [109]:
# original_segm_liver = np.where(original_segm==1,1,0)
# idx = []
# sum_liver = []
# for i in range(0,296,2):
#     idx.append(idx)
#     sum_liver.append(np.sum(original_segm_liver[:,:,i]))
#     plt.imshow(original_segm_liver[:,:,i])
#     plt.show()

In [137]:
original_segm.shape

(296, 512, 512)

In [158]:
img_path = '/vast/amr10211/deep_learning_final_project/images/'
slice_df = image_df[image_df['series_id']==50282]

original_segm = load_total_segmentation_volume('/vast/amr10211/deep_learning_final_project/segmentations/50282.nii')


for i in range(0, 296): 
    print(i)
    
    # # Display image
    # plt.imshow(img[i,:,:])
    # plt.title('Image')
    # plt.show()
    
    img_slice = slice_df.iloc[i,0]
    img = np.array(Image.open(os.path.join(img_path, img_slice)))
    plt.imshow(img)
    plt.title('Image')
    plt.show()
    
    # Display corresponding original segmentation every other time
    # if i % 2 == 0:
    plt.imshow(original_segm[i,:,:])
    plt.title('Original Segm')
    print(np.unique(original_segm[i,:,:]))
    plt.show()
    
    # # Display segmentation
    # plt.imshow(segm[i,:,:])
    # plt.title('Liver Segm')
    # print(np.unique(segm[i,:,:]))
    # plt.show()

FileNotFoundError: No such file or no access: '/vast/amr10211/deep_learning_final_project/segmentations/50282.nii'

In [ ]:
# 